In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [2]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [3]:
# 날짜 처리
data = pd.read_csv('201901-202003.csv')
data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [4]:
# 데이터 정제
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [6]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [7]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
x = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
y = np.log1p(train_num['AMT'])

In [8]:
k = int(len(x)*0.9)

In [9]:
x_train = x[:k]
y_train = y[:k]
x_val = x[k:]
y_val = y[k:]

In [10]:
import xgboost as xgb

In [31]:
train_ds = xgb.DMatrix(x_train, label=y_train)
val_ds = xgb.DMatrix(x_val, label=y_val)
watchlist = [(val_ds, 'eval'), (train_ds, 'train')]

In [21]:
params = {
            'n_estimators'=5000,
            'max_depth': 3,
            'learning_rate' : 0.05,
            'objective': 'reg:tweedie',
            'loss_function': 'rmse',
            'eta' : 1,
            'lambda_l2' : 0.1
        }

In [33]:
model = xgb.train(params,
                  train_ds,5000,
                  watchlist
                  )

[23:14:44] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { lambda_l2, loss_function } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	eval-tweedie-nloglik@1.5:39.97681	train-tweedie-nloglik@1.5:39.97499
[1]	eval-tweedie-nloglik@1.5:38.30553	train-tweedie-nloglik@1.5:38.30391
[2]	eval-tweedie-nloglik@1.5:36.72785	train-tweedie-nloglik@1.5:36.72626
[3]	eval-tweedie-nloglik@1.5:35.23965	train-tweedie-nloglik@1.5:35.23819
[4]	eval-tweedie-nloglik@1.5:33.83676	train-tweedie-nloglik@1.5:33.83540
[5]	eval-tweedie-nloglik@1.5:32.51545	train-tweedie-nloglik@1.5:32.51424
[6]	eval-tweedie-nloglik@1.5:31.27187	train-tweedie-nloglik@1.5:31.27069
[7]	eval-tweedie-nloglik@1.5:30.10258	train-tweedie-nloglik@1.5:30.10148
[8]	eval-tweed

[106]	eval-tweedie-nloglik@1.5:15.09601	train-tweedie-nloglik@1.5:15.09580
[107]	eval-tweedie-nloglik@1.5:15.09584	train-tweedie-nloglik@1.5:15.09563
[108]	eval-tweedie-nloglik@1.5:15.09571	train-tweedie-nloglik@1.5:15.09549
[109]	eval-tweedie-nloglik@1.5:15.09560	train-tweedie-nloglik@1.5:15.09538
[110]	eval-tweedie-nloglik@1.5:15.09543	train-tweedie-nloglik@1.5:15.09521
[111]	eval-tweedie-nloglik@1.5:15.09533	train-tweedie-nloglik@1.5:15.09511
[112]	eval-tweedie-nloglik@1.5:15.09526	train-tweedie-nloglik@1.5:15.09503
[113]	eval-tweedie-nloglik@1.5:15.09515	train-tweedie-nloglik@1.5:15.09493
[114]	eval-tweedie-nloglik@1.5:15.09505	train-tweedie-nloglik@1.5:15.09483
[115]	eval-tweedie-nloglik@1.5:15.09496	train-tweedie-nloglik@1.5:15.09474
[116]	eval-tweedie-nloglik@1.5:15.09489	train-tweedie-nloglik@1.5:15.09467
[117]	eval-tweedie-nloglik@1.5:15.09475	train-tweedie-nloglik@1.5:15.09454
[118]	eval-tweedie-nloglik@1.5:15.09470	train-tweedie-nloglik@1.5:15.09449
[119]	eval-tweedie-nlogli

[216]	eval-tweedie-nloglik@1.5:15.09048	train-tweedie-nloglik@1.5:15.09033
[217]	eval-tweedie-nloglik@1.5:15.09046	train-tweedie-nloglik@1.5:15.09031
[218]	eval-tweedie-nloglik@1.5:15.09045	train-tweedie-nloglik@1.5:15.09029
[219]	eval-tweedie-nloglik@1.5:15.09043	train-tweedie-nloglik@1.5:15.09028
[220]	eval-tweedie-nloglik@1.5:15.09041	train-tweedie-nloglik@1.5:15.09026
[221]	eval-tweedie-nloglik@1.5:15.09038	train-tweedie-nloglik@1.5:15.09024
[222]	eval-tweedie-nloglik@1.5:15.09035	train-tweedie-nloglik@1.5:15.09021
[223]	eval-tweedie-nloglik@1.5:15.09034	train-tweedie-nloglik@1.5:15.09020
[224]	eval-tweedie-nloglik@1.5:15.09030	train-tweedie-nloglik@1.5:15.09016
[225]	eval-tweedie-nloglik@1.5:15.09029	train-tweedie-nloglik@1.5:15.09015
[226]	eval-tweedie-nloglik@1.5:15.09026	train-tweedie-nloglik@1.5:15.09012
[227]	eval-tweedie-nloglik@1.5:15.09020	train-tweedie-nloglik@1.5:15.09006
[228]	eval-tweedie-nloglik@1.5:15.09016	train-tweedie-nloglik@1.5:15.09002
[229]	eval-tweedie-nlogli

[326]	eval-tweedie-nloglik@1.5:15.08812	train-tweedie-nloglik@1.5:15.08802
[327]	eval-tweedie-nloglik@1.5:15.08809	train-tweedie-nloglik@1.5:15.08799
[328]	eval-tweedie-nloglik@1.5:15.08800	train-tweedie-nloglik@1.5:15.08789
[329]	eval-tweedie-nloglik@1.5:15.08799	train-tweedie-nloglik@1.5:15.08788
[330]	eval-tweedie-nloglik@1.5:15.08799	train-tweedie-nloglik@1.5:15.08787
[331]	eval-tweedie-nloglik@1.5:15.08795	train-tweedie-nloglik@1.5:15.08784
[332]	eval-tweedie-nloglik@1.5:15.08794	train-tweedie-nloglik@1.5:15.08783
[333]	eval-tweedie-nloglik@1.5:15.08790	train-tweedie-nloglik@1.5:15.08780
[334]	eval-tweedie-nloglik@1.5:15.08788	train-tweedie-nloglik@1.5:15.08778
[335]	eval-tweedie-nloglik@1.5:15.08786	train-tweedie-nloglik@1.5:15.08777
[336]	eval-tweedie-nloglik@1.5:15.08784	train-tweedie-nloglik@1.5:15.08774
[337]	eval-tweedie-nloglik@1.5:15.08784	train-tweedie-nloglik@1.5:15.08773
[338]	eval-tweedie-nloglik@1.5:15.08783	train-tweedie-nloglik@1.5:15.08773
[339]	eval-tweedie-nlogli

[436]	eval-tweedie-nloglik@1.5:15.08647	train-tweedie-nloglik@1.5:15.08635
[437]	eval-tweedie-nloglik@1.5:15.08647	train-tweedie-nloglik@1.5:15.08634
[438]	eval-tweedie-nloglik@1.5:15.08645	train-tweedie-nloglik@1.5:15.08633
[439]	eval-tweedie-nloglik@1.5:15.08644	train-tweedie-nloglik@1.5:15.08633
[440]	eval-tweedie-nloglik@1.5:15.08643	train-tweedie-nloglik@1.5:15.08631
[441]	eval-tweedie-nloglik@1.5:15.08642	train-tweedie-nloglik@1.5:15.08630
[442]	eval-tweedie-nloglik@1.5:15.08641	train-tweedie-nloglik@1.5:15.08629
[443]	eval-tweedie-nloglik@1.5:15.08640	train-tweedie-nloglik@1.5:15.08628
[444]	eval-tweedie-nloglik@1.5:15.08639	train-tweedie-nloglik@1.5:15.08627
[445]	eval-tweedie-nloglik@1.5:15.08639	train-tweedie-nloglik@1.5:15.08627
[446]	eval-tweedie-nloglik@1.5:15.08637	train-tweedie-nloglik@1.5:15.08626
[447]	eval-tweedie-nloglik@1.5:15.08637	train-tweedie-nloglik@1.5:15.08625
[448]	eval-tweedie-nloglik@1.5:15.08635	train-tweedie-nloglik@1.5:15.08624
[449]	eval-tweedie-nlogli

...

[4550]	eval-tweedie-nloglik@1.5:15.07012	train-tweedie-nloglik@1.5:15.07014
[4551]	eval-tweedie-nloglik@1.5:15.07013	train-tweedie-nloglik@1.5:15.07014
[4552]	eval-tweedie-nloglik@1.5:15.07012	train-tweedie-nloglik@1.5:15.07015
[4553]	eval-tweedie-nloglik@1.5:15.07012	train-tweedie-nloglik@1.5:15.07015
[4554]	eval-tweedie-nloglik@1.5:15.07012	train-tweedie-nloglik@1.5:15.07015
[4555]	eval-tweedie-nloglik@1.5:15.07012	train-tweedie-nloglik@1.5:15.07014
[4556]	eval-tweedie-nloglik@1.5:15.07012	train-tweedie-nloglik@1.5:15.07014
[4557]	eval-tweedie-nloglik@1.5:15.07012	train-tweedie-nloglik@1.5:15.07014
[4558]	eval-tweedie-nloglik@1.5:15.07012	train-tweedie-nloglik@1.5:15.07014
[4559]	eval-tweedie-nloglik@1.5:15.07012	train-tweedie-nloglik@1.5:15.07014
[4560]	eval-tweedie-nloglik@1.5:15.07011	train-tweedie-nloglik@1.5:15.07014
[4561]	eval-tweedie-nloglik@1.5:15.07012	train-tweedie-nloglik@1.5:15.07014
[4562]	eval-tweedie-nloglik@1.5:15.07011	train-tweedie-nloglik@1.5:15.07014
[4563]	eval-

[4658]	eval-tweedie-nloglik@1.5:15.07002	train-tweedie-nloglik@1.5:15.07005
[4659]	eval-tweedie-nloglik@1.5:15.07002	train-tweedie-nloglik@1.5:15.07004
[4660]	eval-tweedie-nloglik@1.5:15.07002	train-tweedie-nloglik@1.5:15.07004
[4661]	eval-tweedie-nloglik@1.5:15.07001	train-tweedie-nloglik@1.5:15.07004
[4662]	eval-tweedie-nloglik@1.5:15.07001	train-tweedie-nloglik@1.5:15.07004
[4663]	eval-tweedie-nloglik@1.5:15.07001	train-tweedie-nloglik@1.5:15.07004
[4664]	eval-tweedie-nloglik@1.5:15.07001	train-tweedie-nloglik@1.5:15.07003
[4665]	eval-tweedie-nloglik@1.5:15.07001	train-tweedie-nloglik@1.5:15.07003
[4666]	eval-tweedie-nloglik@1.5:15.07001	train-tweedie-nloglik@1.5:15.07003
[4667]	eval-tweedie-nloglik@1.5:15.07001	train-tweedie-nloglik@1.5:15.07003
[4668]	eval-tweedie-nloglik@1.5:15.07000	train-tweedie-nloglik@1.5:15.07003
[4669]	eval-tweedie-nloglik@1.5:15.07000	train-tweedie-nloglik@1.5:15.07003
[4670]	eval-tweedie-nloglik@1.5:15.07001	train-tweedie-nloglik@1.5:15.07004
[4671]	eval-

[4766]	eval-tweedie-nloglik@1.5:15.06984	train-tweedie-nloglik@1.5:15.06987
[4767]	eval-tweedie-nloglik@1.5:15.06984	train-tweedie-nloglik@1.5:15.06987
[4768]	eval-tweedie-nloglik@1.5:15.06984	train-tweedie-nloglik@1.5:15.06987
[4769]	eval-tweedie-nloglik@1.5:15.06984	train-tweedie-nloglik@1.5:15.06987
[4770]	eval-tweedie-nloglik@1.5:15.06984	train-tweedie-nloglik@1.5:15.06987
[4771]	eval-tweedie-nloglik@1.5:15.06984	train-tweedie-nloglik@1.5:15.06987
[4772]	eval-tweedie-nloglik@1.5:15.06984	train-tweedie-nloglik@1.5:15.06987
[4773]	eval-tweedie-nloglik@1.5:15.06984	train-tweedie-nloglik@1.5:15.06987
[4774]	eval-tweedie-nloglik@1.5:15.06984	train-tweedie-nloglik@1.5:15.06987
[4775]	eval-tweedie-nloglik@1.5:15.06984	train-tweedie-nloglik@1.5:15.06987
[4776]	eval-tweedie-nloglik@1.5:15.06983	train-tweedie-nloglik@1.5:15.06987
[4777]	eval-tweedie-nloglik@1.5:15.06983	train-tweedie-nloglik@1.5:15.06987
[4778]	eval-tweedie-nloglik@1.5:15.06983	train-tweedie-nloglik@1.5:15.06987
[4779]	eval-

[4874]	eval-tweedie-nloglik@1.5:15.06975	train-tweedie-nloglik@1.5:15.06979
[4875]	eval-tweedie-nloglik@1.5:15.06975	train-tweedie-nloglik@1.5:15.06979
[4876]	eval-tweedie-nloglik@1.5:15.06975	train-tweedie-nloglik@1.5:15.06979
[4877]	eval-tweedie-nloglik@1.5:15.06975	train-tweedie-nloglik@1.5:15.06979
[4878]	eval-tweedie-nloglik@1.5:15.06975	train-tweedie-nloglik@1.5:15.06979
[4879]	eval-tweedie-nloglik@1.5:15.06975	train-tweedie-nloglik@1.5:15.06979
[4880]	eval-tweedie-nloglik@1.5:15.06975	train-tweedie-nloglik@1.5:15.06979
[4881]	eval-tweedie-nloglik@1.5:15.06975	train-tweedie-nloglik@1.5:15.06979
[4882]	eval-tweedie-nloglik@1.5:15.06975	train-tweedie-nloglik@1.5:15.06979
[4883]	eval-tweedie-nloglik@1.5:15.06974	train-tweedie-nloglik@1.5:15.06978
[4884]	eval-tweedie-nloglik@1.5:15.06974	train-tweedie-nloglik@1.5:15.06978
[4885]	eval-tweedie-nloglik@1.5:15.06974	train-tweedie-nloglik@1.5:15.06978
[4886]	eval-tweedie-nloglik@1.5:15.06974	train-tweedie-nloglik@1.5:15.06978
[4887]	eval-

[4982]	eval-tweedie-nloglik@1.5:15.06965	train-tweedie-nloglik@1.5:15.06970
[4983]	eval-tweedie-nloglik@1.5:15.06965	train-tweedie-nloglik@1.5:15.06970
[4984]	eval-tweedie-nloglik@1.5:15.06965	train-tweedie-nloglik@1.5:15.06970
[4985]	eval-tweedie-nloglik@1.5:15.06965	train-tweedie-nloglik@1.5:15.06970
[4986]	eval-tweedie-nloglik@1.5:15.06965	train-tweedie-nloglik@1.5:15.06970
[4987]	eval-tweedie-nloglik@1.5:15.06965	train-tweedie-nloglik@1.5:15.06970
[4988]	eval-tweedie-nloglik@1.5:15.06965	train-tweedie-nloglik@1.5:15.06970
[4989]	eval-tweedie-nloglik@1.5:15.06965	train-tweedie-nloglik@1.5:15.06970
[4990]	eval-tweedie-nloglik@1.5:15.06965	train-tweedie-nloglik@1.5:15.06969
[4991]	eval-tweedie-nloglik@1.5:15.06964	train-tweedie-nloglik@1.5:15.06969
[4992]	eval-tweedie-nloglik@1.5:15.06964	train-tweedie-nloglik@1.5:15.06969
[4993]	eval-tweedie-nloglik@1.5:15.06964	train-tweedie-nloglik@1.5:15.06969
[4994]	eval-tweedie-nloglik@1.5:15.06964	train-tweedie-nloglik@1.5:15.06969
[4995]	eval-

In [34]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=x.columns)

In [36]:
# 예측
temp1 = xgb.DMatrix(temp)
pred = model.predict(temp1)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [37]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [38]:
# 제출 파일 만들기
submission = pd.read_csv('submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('XGB5000_submission.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,2.010114e+08
1,202004,강원,골프장 운영업,1.483717e+09
2,202004,강원,과실 및 채소 소매업,6.185174e+08
3,202004,강원,관광 민예품 및 선물용품 소매업,5.249193e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.275275e+07
